### Pregunta 2

Después de haber instalado MPI procedemos a importarlo así como las otras funciones que nos puedans servir como numpy

In [72]:
#from mpi4py import MPI
#import numpy as np
#import random
import pandas as pd

Para comprobar que MPI se instaló correctamente, usamos el ejemplo visto en clase.

In [13]:
%%writefile mpi_example1.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD # all available processors to communicate 
rank = comm.Get_rank() # give ranks to processors, local variable for e/ processor
size = comm.Get_size() # each processor identifies the total processors 

print("Hello World from rank ", rank, " out of ", size, " processors ")

Overwriting mpi_example1.py


In [14]:
! mpiexec -n 4 python mpi_example1.py

Hello World from rank  0  out of  4  processors 
Hello World from rank  1  out of  4  processors 
Hello World from rank  2  out of  4  processors 
Hello World from rank  3  out of  4  processors 


a.Que un procesador genere dos numpy array diferentes, cada uno de 1,000,000 observaciones. Llamar “num1” y “num2” a estos numpy

Con nuestro procesador de rango 0 generamos dos arrays de números aleatorios con 1 000 000 de observaciones cada uno que llamaremos "num1" y "num2"

In [36]:
random.seed (1102)

In [46]:
%%writefile numpys.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    print (num1,num2)


Overwriting numpys.py


In [48]:
! mpiexec -n 1 python numpys.py

[0.87890393 0.23608039 0.25866743 ... 0.74521277 0.9093864  0.99825781] [0.01619609 0.31208514 0.75631335 ... 0.26744182 0.76172791 0.79247752]


b.	Enviar cada numpy a un procesador diferente. Que cada uno de los otros procesadores reciba su numpy

Procedemos a enviar "num1" al procesador de rango 1 y "num2" al procesador de rango 2

In [61]:
%%writefile numpys2.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    comm.Send(num1, dest=1)
    comm.Send(num2, dest=2)
elif rank == 1:
    num1 = np.empty(1000000)
    comm.Recv(num1, source=0)
    print (num1)
elif rank == 2:
    num2 = np.empty(1000000)
    comm.Recv(num2, source=0)
    print (num2)

Overwriting numpys2.py


In [62]:
! mpiexec -n 3 python numpys2.py

[0.54972414 0.13346457 0.33676119 ... 0.99672326 0.00520165 0.62373786]
[0.58985467 0.60038454 0.17469465 ... 0.84140889 0.05707542 0.99939542]


c.	Que otro procesador (que no haya recibido nada) reciba “num1” y “num2” y los imprima.

Procedemos a enviar "num1" desde nuestro procesador de rango 1 y "num2" desde nuestro procesador de rango 2 a nuestro procesador de rango 3. 

In [68]:
%%writefile parte2_2.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    comm.Send(num1, dest=1)
    comm.Send(num2, dest=2)
elif rank == 1:
    num1 = np.empty(1000000)
    comm.Recv(num1, source=0)
    print (num1)
elif rank == 2:
    num2 = np.empty(1000000)
    comm.Recv(num2, source=0)
    print (num2)

Writing parte2_2.py


In [69]:
! mpiexec -n 4 python parte2_2.py

[0.91294557 0.08002201 0.44396812 ... 0.14826613 0.87352387 0.22030702]
[0.36044107 0.58287401 0.28592556 ... 0.93334727 0.01238815 0.27647252]


d.	Ejecute el código creado y registre el tiempo que toma realizar este ejercicio.

Procedemos a ejecutar nuestro último código tomando el tiempo de ejecución. 

In [71]:
import time
start = time.time()
! mpiexec -n 4 python parte2_2.py
end = time.time()
print(end - start)

[0.91259605 0.70382155 0.0011705  ... 0.01362625 0.87246215 0.29451386]
[0.65523925 0.93751911 0.93899528 ... 0.7779723  0.90110366 0.86416309]
0.7058086395263672


e.	En otro chunk responda: ¿existe una manera de agilizar este proceso con las herramientas de MPI? Sea detallado en su respuesta y argumentos.

### Pregunta 3

Generar un numpy array que almacene el archivo: “tarea2.csv”. 

In [67]:
from numpy import genfromtxt
tarea2 = genfromtxt('tarea2.csv', delimiter=',')
print(tarea2.shape)

OSError: tarea2.csv not found.

a.	Escribir un código que halle el valor máximo de “tarea2” usando un procesador. Imprimir el valor máximo. Registrar el tiempo de demora.

In [ ]:
%%writefile p3a.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    maximo = tarea2.max()
    print(maximo)

In [ ]:
import time
start = time.time()
! mpiexec -n 1 python p3a.py
end = time.time()
print(end - start)

b.	Escribir un código que realice las siguientes indicaciones. Dividir el numpy en dos partes iguales. Que dos procesadores distintos encuentren el máximo de cada parte. Que otro procesador junte los máximos hallados y encuentre el máximo global. Este resultado debe ser igual al de 3a. Registrar el tiempo de demora.

In [ ]:
tarea2_chunk = []
for chunk in pd.read_csv(tarea2 , chunks=2):
    chunks.append(tarea2)

In [ ]:
%%writefile p3b.py 
from mpi4py import MPI

import numpy as np
import pandas as pd
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

tarea2_chunk = np.array_split(tarea2, 2)

if rank == 0:
    maximo = tarea2.max()
    print(maximo)

c.	Repetir 3b dividiendo el numpy original en tres partes. Registrar el tiempo de demora.

d.	Comparar los tiempos registrados en 3a, 3b y 3c. ¿Hay una reducción del tiempo? ¿La reducción del tiempo es lineal? ¿Por qué?